In [1]:
import pandas as pd
file_path='../data/creditcard.csv'
df=pd.read_csv(file_path)
print("top five rows of the data: ") 
display(df.head())
print("\n DATASET INFORMATION:")
df.info

top five rows of the data: 


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0



 DATASET INFORMATION:


<bound method DataFrame.info of             Time         V1         V2        V3        V4        V5  \
0            0.0  -1.359807  -0.072781  2.536347  1.378155 -0.338321   
1            0.0   1.191857   0.266151  0.166480  0.448154  0.060018   
2            1.0  -1.358354  -1.340163  1.773209  0.379780 -0.503198   
3            1.0  -0.966272  -0.185226  1.792993 -0.863291 -0.010309   
4            2.0  -1.158233   0.877737  1.548718  0.403034 -0.407193   
...          ...        ...        ...       ...       ...       ...   
284802  172786.0 -11.881118  10.071785 -9.834783 -2.066656 -5.364473   
284803  172787.0  -0.732789  -0.055080  2.035030 -0.738589  0.868229   
284804  172788.0   1.919565  -0.301254 -3.249640 -0.557828  2.630515   
284805  172788.0  -0.240440   0.530483  0.702510  0.689799 -0.377961   
284806  172792.0  -0.533413  -0.189733  0.703337 -0.506271 -0.012546   

              V6        V7        V8        V9  ...       V21       V22  \
0       0.462388  0.239599  

In [2]:
class_counts=df['Class'].value_counts()
fraud_percentage=(class_counts[1]/class_counts.sum())*100
print(f"amount of legitimate transaction(Class 0): {class_counts[0]}")
print(f"amount of fraudulent transaction(Class 1): {class_counts[1]}")
print(f"percentage of transactions that are fraudulent: {fraud_percentage:.4f}%")
                  
                  


amount of legitimate transaction(Class 0): 284315
amount of fraudulent transaction(Class 1): 492
percentage of transactions that are fraudulent: 0.1727%


In [3]:
x=df.drop('Class',axis=1)
y=df['Class']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(
    x,y,test_size=0.3,random_state=42,stratify=y
)
print("shape of x_train:",x_train.shape)
print("shape of x_test:",x_test.shape)
print("shape of y_train:",y_train.shape)
print("shape of y_test:",y_test.shape)



shape of x_train: (199364, 30)
shape of x_test: (85443, 30)
shape of y_train: (199364,)
shape of y_test: (85443,)


In [4]:
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression


scaler = StandardScaler()


x_train_scaled = scaler.fit_transform(x_train)


x_test_scaled = scaler.transform(x_test)



smote = SMOTE(random_state=42)
x_train_resampled, y_train_resampled = smote.fit_resample(x_train_scaled, y_train)



model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(x_train_resampled, y_train_resampled)

print("Model training complete on scaled and resampled data!")

Model training complete on scaled and resampled data!


In [5]:
from sklearn.metrics import classification_report, confusion_matrix

# Make predictions on the scaled test data
y_pred = model.predict(x_test_scaled)

# Print the classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     85295
           1       0.06      0.88      0.12       148

    accuracy                           0.98     85443
   macro avg       0.53      0.93      0.55     85443
weighted avg       1.00      0.98      0.99     85443

Confusion Matrix:
[[83410  1885]
 [   18   130]]


In [6]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix


xgb_model = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')


print("Training the XGBoost model...")
xgb_model.fit(x_train_resampled, y_train_resampled)
print("Training complete.")


y_pred_xgb = xgb_model.predict(x_test_scaled)


print("\nXGBoost Classification Report:")
print(classification_report(y_test, y_pred_xgb))

print("\nXGBoost Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_xgb))

Training the XGBoost model...


C:\mlops project\venv\lib\site-packages\xgboost\training.py:183: UserWarning: [14:19:21] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Training complete.

XGBoost Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85295
           1       0.72      0.82      0.77       148

    accuracy                           1.00     85443
   macro avg       0.86      0.91      0.88     85443
weighted avg       1.00      1.00      1.00     85443


XGBoost Confusion Matrix:
[[85249    46]
 [   27   121]]


In [7]:
import joblib
import os


os.makedirs('models', exist_ok=True)


model_path = 'models/xgboost_model.joblib'
scaler_path = 'models/scaler.joblib'


joblib.dump(xgb_model, model_path)


joblib.dump(scaler, scaler_path)

print(f"Model saved to {model_path}")
print(f"Scaler saved to {scaler_path}")

Model saved to models/xgboost_model.joblib
Scaler saved to models/scaler.joblib


In [8]:
# Select the first row from the original (unscaled) test set
sample = x_test.iloc[0]

# Convert it to the JSON format
print(sample.to_json())

{"Time":127294.0,"V1":-2.5373312409,"V2":1.8908639895,"V3":-0.8405545388,"V4":-1.1027585745,"V5":-2.105724965,"V6":0.3678108232,"V7":-2.7378443185,"V8":-3.5433138804,"V9":-0.4831465855,"V10":0.1127240821,"V11":-1.6125864643,"V12":0.6358552717,"V13":0.4068711504,"V14":0.8543712686,"V15":0.5057533823,"V16":-0.9477880926,"V17":0.5573310971,"V18":1.5466382651,"V19":-0.7890474133,"V20":-1.9596112476,"V21":4.3281648156,"V22":-2.1673294995,"V23":1.0225672506,"V24":-0.2411432745,"V25":-1.0948853669,"V26":-0.4754274375,"V27":-0.1771570437,"V28":-0.0826905454,"Amount":37.4}
